In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from torch_geometric.transforms import RandomLinkSplit
import pandas as pd

/home/acaia/Analysa/Analysa/.analysa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, aggr='mean'):
        super(Encoder, self).__init__()
        self.conv1 = pyg_nn.SAGEConv(in_channels, 2 * out_channels, aggr=aggr) # Eventually change layer type
        self.batch1 = pyg_nn.BatchNorm(2 * out_channels)
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = pyg_nn.SAGEConv(2 * out_channels, out_channels,  aggr=aggr) # Eventually change layer type
        self.batch2 = pyg_nn.BatchNorm(out_channels)

    def forward(self, x, edge_index): # Modify to adapt to the dataset complexity
        x = self.conv1(x, edge_index)
        x = self.batch1(x)
        x = self.drop1(x)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.batch2(x)

        return x
    
class Decoder(nn.Module):
    def __init__(self, in_channels):
        super(Decoder, self).__init__()
        self.linear = nn.Linear(in_channels, 1)

    def forward(self, z, edge_index):
        # By default, it computes the inner product of the node embeddings
        adj = torch.matmul(z[edge_index[0]], z[edge_index[1]].t())
        return adj[edge_index[0], edge_index[1]]
    
# Defining traing and evaluation functions
def train(epoch):
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_data.edge_index.to(device))
    loss = model.recon_loss(z, train_data.pos_edge_label_index.to(device))
    loss.backward()
    optimizer.step()

def test(pos_edge_index, neg_edge_index):
    model.eval()
    # Encode the node features to get the embeddings
    with torch.no_grad():
        z = model.encode(x, train_data.edge_index.to(device))
    
    # Compute the reconstruction loss
    return model.test(z, pos_edge_index.to(device), neg_edge_index.to(device))
    

In [3]:
# Model definition
import os
LATENT_DIM = 32
data_path = os.path.join('..', 'data', 'graph_data.pt')
dataset = torch.load(data_path, weights_only=False)
data = dataset['data']

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Instantiate the customized encoder in GAE model
transform = RandomLinkSplit(is_undirected=True, split_labels=True,add_negative_train_samples=True)
train_data, val_data, test_data = transform(data)

model = pyg_nn.GAE(Encoder(data.num_features, LATENT_DIM)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x = train_data.x.to(device)

Using device: cuda


In [4]:
# Training loop
for epoch in range(1, 1000):
    loss = train(epoch)
    # AUC tells how well the model is able to distinguish real nodes from fake ones
    # AP - Average Precision is the complementary metric of AUC 
    auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)

    if epoch % 10 == 0:
        print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 010, AUC: 0.6211, AP: 0.6234
Epoch: 020, AUC: 0.7824, AP: 0.7604
Epoch: 030, AUC: 0.8189, AP: 0.7987
Epoch: 040, AUC: 0.8277, AP: 0.8043
Epoch: 050, AUC: 0.8319, AP: 0.8069
Epoch: 060, AUC: 0.8364, AP: 0.8092
Epoch: 070, AUC: 0.8350, AP: 0.8075
Epoch: 080, AUC: 0.8328, AP: 0.8067
Epoch: 090, AUC: 0.8309, AP: 0.8062
Epoch: 100, AUC: 0.8326, AP: 0.8081
Epoch: 110, AUC: 0.8262, AP: 0.8082
Epoch: 120, AUC: 0.8154, AP: 0.8007
Epoch: 130, AUC: 0.7937, AP: 0.7775
Epoch: 140, AUC: 0.8235, AP: 0.8086
Epoch: 150, AUC: 0.8170, AP: 0.8005
Epoch: 160, AUC: 0.8206, AP: 0.8060
Epoch: 170, AUC: 0.8240, AP: 0.8098
Epoch: 180, AUC: 0.8167, AP: 0.8021
Epoch: 190, AUC: 0.8066, AP: 0.7942
Epoch: 200, AUC: 0.8177, AP: 0.8009
Epoch: 210, AUC: 0.7948, AP: 0.7770
Epoch: 220, AUC: 0.8104, AP: 0.7974
Epoch: 230, AUC: 0.7841, AP: 0.7638
Epoch: 240, AUC: 0.7608, AP: 0.7277
Epoch: 250, AUC: 0.7774, AP: 0.7519
Epoch: 260, AUC: 0.8091, AP: 0.7955
Epoch: 270, AUC: 0.7950, AP: 0.7780
Epoch: 280, AUC: 0.7337, AP:

In [5]:
model.eval()
z = model.encode(x, train_data.edge_index.to(device))
emb_sample = z[0]
print(f'Number of nodes: {z.shape[0]}')
print(f'Number of edges: {train_data.edge_index.shape[1]}')
print(f'Embedding sample: {z}')

Number of nodes: 4681
Number of edges: 20236
Embedding sample: tensor([[ 0.2127,  0.1965,  0.2142,  ...,  0.2129,  0.1164, -0.7988],
        [-0.0487, -0.1120,  0.6771,  ...,  0.1587, -0.3308, -1.1364],
        [-0.0772, -7.5162,  0.5632,  ..., -0.1415, -5.4869, -5.2316],
        ...,
        [-6.0341,  0.1309, -2.1863,  ..., -2.2882, -0.6434,  0.1236],
        [-6.0327,  0.1309, -2.1866,  ..., -2.2895, -0.6445,  0.1243],
        [-6.0341,  0.1309, -2.1863,  ..., -2.2882, -0.6434,  0.1236]],
       device='cuda:0', grad_fn=<NativeBatchNormBackward0>)


In [6]:
mapping_list = dataset['idx2user']
embeddings = z
embeddings_np = embeddings.cpu().detach().numpy()

df = pd.DataFrame(embeddings_np)
df.insert(loc=0, column='id', value=range(len(embeddings_np)))
df.insert(loc=1, column='user_id', value= df['id'].map(mapping_list))

df.to_csv('user_embeddings.csv', index=False)